<h1>Segmentation of best localities in Hyderabad</h1>

<br>
<h3>Introduction</h3>
<p>In this Capstone project I'm using a scenario where a new family comes to Hyderabad for settling. The first problem they face was is which area is best to choose for residency. Hyderabad is a vast city which many localities. So, finding a best area with all the facilities is important. Every person wants a house that is near to hostipals, schools, parks etc. In project helps a person to find the best area for buying a new house.</p>

<br>
<h3>Business Problem</h3>
<p>The objective of this project is to determine the best neighborhood in hyderabad to buy a new house which is near to parks, restaurents, office etc. With the help of clustering, we would be able to determine the best neighborhood.</p>

<br>
<h3>Data</h3>
<p>To solve this problem, we will require following data :</p>
<ul>
    <li>Hyderabad localities list</li>
    <li>Average property rates per sqft in different localities</li>
    <li>Schools, Hospitals, Office near the area</li>
    <li>Places to visit in each area</li>
    <li>Latitude and Longitude of near area</li>
</ul>
    

<br>
<h3>Data Extraction</h3>
<ul>
    <li>Hyderabad localities and property rates from makaan.com using web scrapping</li>
    <li>Nearby schools, hostipal, places to visit in each area using Four Square API</li>
    <li>Latitude and Longitude dataset of near area using Geocoder package</li>
</ul>